In [ ]:
import io
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt 
import json
import pandas as pd
import numpy as np
import plotly
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import math 
import pandas as pd
report=pd.DataFrame({'creteria':['Accuracy','erreur I','erreurII' ,'AUC','CV ACU','AIC','som error']})
import base64
from sklearn import  metrics 
from sklearn.metrics import confusion_matrix
from sklearn import cross_validation
import matplotlib.pylab as plt
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
from loremipsum import get_sentences
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.feature_selection import RFE
from sklearn.externals import joblib
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
import time 


In [7]:
## change the link   
lien="C:/Users/DELL/Desktop/"

In [ ]:

app = dash.Dash()
app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})
app.scripts.config.serve_locally=True
colors = {
    'background': '#f2f3f4',
    'text': '#ee0c35'
}

app.layout = html.Div(style={'backgroundColor': colors['background']},
    children=[
    html.H1(
        children='Ooreedoo',
        style={
                'textAlign': 'center',
                'color': colors['text']
            }
    ),
    html.Img(src=lien+'ooredoo.png'),

    html.Div(children='Dash: A web application framework for Machine Learning .',
             style={
            'textAlign': 'center',
            'color': colors['text']}
        ),
        html.H5("Upload Files"),
        dcc.Upload(
            id='upload-data',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select Files')
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px',
                'position': 'static',
                'top': '120px', 'left': '20%',
                'backgroundColor': '#ccffff'
            },
            multiple=False),
#################################################################################################################        
###### Data Explore
        
        html.Div([html.H4("Data Explore") ],
                 style={'textAlign': 'center',
                        'fontSize':'25px',
                        'textAlign': 'center',
                        'color': colors['text']
                       }
                ),
        html.Div([
            html.Div([
                html.H5("Uploaded Data"),
                html.Div(dt.DataTable(rows=[{}], id='table',row_selectable=True, filterable=True,
                sortable=True,)),
            ],className="six columns"),
            
            html.Div([
                html.H5('Data Summary'), 
                html.Div(dt.DataTable(rows=[{}], id='summary',row_selectable=True, filterable=True,
                sortable=True, row_height=30,)),
            ],className="six columns"),
            
            ],className="row",
            style={
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'backgroundColor' : '#e6f0ff',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
            }
        ),
#################################################################################################################        
#### select  columns 
        
        html.Br(),
        html.Div([html.H4("Select columns") ],
                 style={'textAlign': 'center',
                        'fontSize':'25px',
                        'textAlign': 'center',
                        'color': colors['text']
                       }
                ),
        html.Div(

            [
                 html.Div([
                     html.Div([
                         html.H6("Select predictive feature"),
                         dcc.Dropdown(id='dropdownY',multi = False, 
                                       placeholder='Select dependant var'), 
                         html.Div('Note: depente variable will be removed automaticly '),
                         html.Br(),
                         html.H6("Select id sbscriber's feature"),        
                          dcc.Dropdown(id='subs',
                                       multi = False,
                                       placeholder='Select subscriber var'), 
                         
                     ],className="six columns"),
                     ##
                     html.Div([
                         dcc.RadioItems(id='selectAll',options=[
                             {'label': 'select all features', 'value': 'all'},
                             {'label': 'select manually', 'value': 'notAll'},]
                                        ,value= None),
                    html.H5("Select explicative features manually"),        
                    dcc.Dropdown(id='dropdownX',
                        multi = True,
                        placeholder='Select explicative var'),

                     ],className="six columns"),

                     
                 ],className="row"),
                html.Div([
                     html.Br(),
                     html.H6("Select percentage of splitting Data into: train and  test"), 
                     html.Div(dcc.Slider(id='echSplit',  min=0.01, max=1,step=0.05, value=0.3,),
                              style={'width': '20%','display': 'inline-block'},),
                              html.Div(id='output-container-range-slider'),
                     ]),
            
            ],style={
                'borderRadius': '5px',
                'backgroundColor' : '#e6ffe6',
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
            }
        ) , 

#################################################################################################################        
############### Features selection
        
        html.Div(children='Features selection', style={
            'fontSize':'25px',
            'textAlign': 'center',
            'color': colors['text']
        }),
        
        html.Div([
            html.Div([
                html.Div(children='Features selection criteria', style={
                    'textAlign': 'center',
                    'fontSize':'17px',
                }),
                dcc.RadioItems(id='selectVar',options=[
                    {'label': 'RFE: Recursive Feature Elimination', 'value': 'RFE'},
                    {'label': 'Delete Correlated Features', 'value': 'corr'},]
                               ,value= None),
                html.Br(),
                html.Br(),
                html.Div([html.Div('Pick a classifier for RFE')], style={'fontSize':'15px'}),
                dcc.RadioItems(id='modelRFE',options=[
                    {'label': 'Gradient Boosting', 'value': 'gbm'},
                    {'label': 'Random Forest', 'value': 'random'},
                    {'label': 'Decision Tree', 'value': 'DT'},
                    {'label': 'Naive Bays', 'value': 'bays'},
                    {'label': 'Regression Logistic', 'value': 'logreg'},
                ],value= None),
            
                html.Br(),
                html.Br(),
                html.Div([
                    html.Div([html.Div('Correlation threshold')], style={'fontSize':'15px'}),
                    dcc.Slider(id='seuil',  min=0.01, max=1,step=0.05, value=0.7,),
                    html.Div(id='seuilOut'),
                ],style={'width': '30%'}),
                    
                
                html.Button('Generate selection', id='buttonSelect' ),
            ],className="six columns"),
            ###
            html.Div([
                html.Div(children='Results', style={
                    'textAlign': 'center',
                    'fontSize':'17px',
                }),
                #html.Div(id='resultSelection', children='Numbre of selected Features'),
                html.Div(dt.DataTable(rows=[{}], id='tableBestFeat',row_selectable=True, filterable=True,
                sortable=True,column_widths=[110, 110], row_height=28,)),
               
            ],className="six columns"),
            html.Div([

                html.Div('Do you want to use the selected features ?'),                
                dcc.RadioItems(id='useSelectedFeature',options=[
                    {'label': 'Yes', 'value': 'yes'},
                    {'label': 'No', 'value': 'no'},
                ],value= None),                
                
            ],style={
                'textAlign': 'center',
                'fontSize':'18px',}
            )

        ],className="row",
            style={
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'backgroundColor' : '#e6f0ff',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
            }

            
        ), 
        
        

#################################################################################################################        
##############    select classifier   #########

        html.Div(children='Select a classifier', style={
            'fontSize':'25px',
            'textAlign': 'center',
            'color': colors['text']
        }),
        ###############
        html.Div([
            html.Div([dcc.RadioItems(id='model',
                                     options=[
                                         {'label': 'mlp classifier', 'value':'mlp'},
                                         {'label': 'mlp classifier with ACP ', 'value':'mlpACP'},
                                         {'label': 'Gradient Boosting', 'value': 'gbm'},
                                         {'label': 'AdaBoost', 'value':'adaBoost'},
                                         {'label': 'Bagging Tree Classifier', 'value':'bagging'},
                                         {'label': 'Random Forest', 'value': 'random'},
                                         {'label': 'Decision Tree', 'value':'DT'},
                                         {'label': 'Naive Bays', 'value': 'bays'},
                                         {'label': 'logistic regression classifier', 'value': 'logreg'}
                                     ],labelStyle={'display': 'inline-block'},value= None)],
                     style={'width': '80%',
                            'position': 'static',
                            'marginLeft': 'auto',
                            'marginRight': 'auto',}),
            
            html.Div(children='Select Parameter', 
                     style={'fontSize':'20px',
                            'textAlign': 'center',
                           }),
            ####
            html.Div([
                html.Div([html.Div([html.Div('MLP')], style={'fontSize':'19px','color':'#00004d'}),
                          html.Div('Default Paratmers:'),dcc.RadioItems(id='defaultMLP',
                                                                options=[{'label': 'Yes', 'value':'T'}
                                                                         ,{'label': 'No', 'value':'F'}
                                                                        ],
                                                                         value='T'  ),                        
                          html.Div('Number of neurals:'),dcc.Input(id='neural', type='text') ,
                          html.Div('Number of layers:'),dcc.Input(id='layers', type='text'),
                          html.Div('Early stop:'),dcc.RadioItems(id='stop',
                                                                options=[{'label': 'True', 'value':'T'}
                                                                         ,{'label': 'False', 'value':'F'}
                                                                        ],
                                                                         value='T')
                         ],className="six columns"),
                #
                html.Div([html.Div([html.Div('Decision Tree,Random Froest, Gradient Boosting')],
                                   style={'fontSize':'19px','color':'#00004d'}),
                          html.Div('Default Paratmers:'),dcc.RadioItems(id='defaultDT',
                                                                options=[{'label': 'Yes', 'value':'T'}
                                                                         ,{'label': 'No', 'value':'F'}
                                                                        ],                          
                                                                         value='T'  ),                        
                          html.Div('number of trees:'),dcc.Input(id='NEstimators', type='text'),
                          html.Div('max depth:'),dcc.Input(id='maxDepth', type='text'),
                          html.Div('Min sample split:'),dcc.Input(id='MinSampleSplit', type='text'),
                          html.Div('Min sample leaf: '),dcc.Input(id='MinSampleLeaf', type='text'),
                          html.Div('Max Feature: '),dcc.RadioItems(id='maxFeature',
                                                                options=[{'label': 'auto :Max Feature =sqrt(n_features)', 'value':'auto'}
                                                                         ,{'label': 'sqrt, Max Feature=sqrt(n_features)', 'value':'sqrt'}
                                                                         ,{'label': 'log2, Max Feature =log2(n_features)', 'value':'sqrt'}
                                                                         ,{'label': 'None, Max Feature =n_features', 'value':'None'}
                                                                        ],
                                                                         value='None')
                         ],className="six columns"),
            ],className="row"),
        ],style={'borderRadius': '5px',
                'backgroundColor' : '#e6ffe6',
                # 'display': 'block',
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
                }
        ),
        html.Br(),


        

#########################################################################################################"        
####### model report 
        
        html.Div(children='Model Report', style={
            'fontSize':'25px',
            'textAlign': 'center',
            'color': colors['text']
        }),
        
        html.Div([
            html.Div([
                html.Div(children='model report', style={
                    'textAlign': 'center',
                }),                
                html.Button( 'Generate model report', id='report'),
                html.Div(dt.DataTable(rows=[{}], id='modelReport',row_selectable=True, filterable=True,
                sortable=True,column_widths=[130, 130], row_height=30,), style ={'align':"center"}),
                html.Br(),
                html.Button('Confusion Matrix',id='ButtonMatrix'),
                html.Div(dt.DataTable(rows=[{}], id='matrix',row_selectable=True, filterable=True,
                sortable=True,column_widths=[160, 160], row_height=50,)),
                html.Div('Note: the column represent the predicted and the row represent the actual'),
                
            ],className="six columns"),
            html.Div([
                html.Div(children='ROC Curve', style={
                    'textAlign': 'center',
                }),
                html.Button('Genrate ROC cuve', id='roc' ),
                dcc.Graph(id='my-graph'),                
            ],className="six columns"),

        ],className="row",
            style={
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'backgroundColor' : '#e6f0ff',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
            }

            
        ), 
        
#########################################################################################################"        
####### compare models
        
        html.Br(),
        html.Div(children='Compare models', style={
            'fontSize':'25px',
            'textAlign': 'center',
            'color': colors['text']
        }), 
        html.Div([
            #
            html.Br(),        
            dcc.Checklist(id='Multimodel',
                          options=[
                              {'label': 'mlp classifier', 'value':'mlp'},
                              {'label': 'mlp classifier with ACP ', 'value':'mlpACP'},
                              {'label': 'Gradient Boosting', 'value': 'gbm'},
                              {'label': 'AdaBoost', 'value':'adaBoost'},
                              {'label': 'Bagging Tree Classifier', 'value':'bagging'},
                              {'label': 'Random Forest', 'value': 'random'},
                              {'label': 'Decision Tree', 'value':'DT'},
                              {'label': 'Naive Bays', 'value': 'bays'},
                              {'label': 'logistic regression classifier', 'value': 'logreg'}
                          ],
                          values=['mlp','gbm'],
                          labelStyle={'display': 'inline-block'},
                          ),
            
            
            html.Br(),        
            html.Button('Genrate multi ROC cuve', id='Mroc'),
            dcc.Graph(id='multiROC'),
            html.Div(dt.DataTable(rows=[{}], id='compareTable',row_selectable=True, filterable=True,
                sortable=True,column_widths=[160, 160], row_height=50,)),
            
        ],
            style={
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'backgroundColor' : '#e6ffe6',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
            }            
        ),

#################################################################################################################        
######## Make a prediction
        
        html.Br(),
        html.Div(children='Make a prediction and download it ', style={
            'fontSize':'25px',
            'textAlign': 'center',
        }), 

        html.Div([
            html.H5("Upload Files"),
            dcc.Upload(
                id='upload_pred',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                },
                multiple=False),


            html.Br(),
            html.Div(dt.DataTable(rows=[{}], id='predict',row_selectable=True, filterable=True,
            sortable=True,column_widths=[100, 100], row_height=30,)), 
            html.Br(),
            html.Div(dcc.Input(id='url', type='text')),
            html.Button('downlaod Prediction', id='download'),
            html.Div(id='downloadResult',
                 children='Enter the path and click to download the prediction table'),
            
        ],
            style={
                'width': '80%',
                'borderStyle': 'solid',
                'borderRadius': '5px',
                'borderColor': '#e1e1d0',
                'backgroundColor' : '#e6f0ff',
                'position': 'static',
                'marginLeft': 'auto',
                'marginRight': 'auto',
                'left': 0,
                'right': 0
            } 
        )

        
    ]
)
#################################################################################################################        
#################################################################################################################        
#################################################################################################################        
#################################################################################################################        
#################################################################################################################        
### upload data

def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')), sep=';')
            print('csv uploaded')
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
            print('xls uploaded')


    except Exception as e:
        print(e)
        return None

    return df
#################

@app.callback(Output('table', 'rows'),
              [Input('upload-data', 'contents'),
               Input('upload-data', 'filename')])
def update_output(contents, filename):
    if contents is not None:
        df = parse_contents(contents, filename)
        if df is not None:
            print('data ulloded  successflly')
            return df.to_dict('records')
        else:
            print(' no upload data none')
            return [{}]
    else:
        print('contenant is none !')
        return [{}]
##################

@app.callback(Output('summary', 'rows'),
              [Input('upload-data', 'contents'),
               Input('upload-data', 'filename')])
def summary (contents, filename):
    if contents is not None:
        df = parse_contents(contents, filename)
        desc=df.describe()
        desc['description'] =desc.index 
        cols = desc.columns.tolist()
        desc = desc[[cols[-1]] + cols[:-1]] 
        return desc.to_dict('records')
    else:
        return [{}]

    
############################################################################################################
################## select columns

@app.callback(Output('dropdownX', 'options'),
              [Input('table', 'rows')])
def update_filter_column_options(tablerows):
    dff = pd.DataFrame(tablerows) # <- problem! dff stays empty even though table was uploaded

    print ("updating... dff empty?:", dff.empty )#result is True, labels stay empty

    return [{'label': i, 'value': i} for i in sorted(list(dff))]

@app.callback(Output('dropdownY', 'options'),
              [Input('table', 'rows')])
def update_filter_column_options(tablerow):
    dfy = pd.DataFrame(tablerow) # <- problem! dff stays empty even though table was uploaded

    print ("updating... dff empty?:", dfy.empty )#result is True, labels stay empty

    return [{'label': i, 'value': i} for i in sorted(list(dfy))]

@app.callback(Output('subs', 'options'),
              [Input('table', 'rows')])
def update_filter_column_options(tablerow):
    df = pd.DataFrame(tablerow) # <- problem! dff stays empty even though table was uploaded

    print ("updating... dff empty?:", df.empty )#result is True, labels stay empty

    return [{'label': i, 'value': i} for i in sorted(list(df))]

#####################
@app.callback(Output('output-container-range-slider', 'children'),
              [Input('echSplit', 'value')])
def select_output_message(value):
    return  'You have selected "{}" %'.format(value*100)

    
############################################################################################################
##################### Feature selection

@app.callback(Output('seuilOut','children'),
              [Input('seuil','value')])
def seuil_output(value):
    return 'You have selected "{}" %'.format(value)

@app.callback(Output('tableBestFeat', 'rows'),
              [Input('table', 'rows'),
               Input('selectVar', 'value'),
               Input('modelRFE', 'value'),
               Input('seuil', 'value'),
               Input('buttonSelect','n_clicks'),
               Input('dropdownX', 'value'),
               Input('dropdownY', 'value'),
               Input('echSplit', 'value'),
               Input ('selectAll', 'value'),
               Input ('subs', 'value'),
              ])
def FeaturesSelection (table,selectVar,modelRFE,seuil,buttonSelect,xvar, yvar, ech,selectAll,subs):
    if (buttonSelect==None): 
        return [{}]
    else:
        df =  pd.DataFrame(table)
        if (selectAll== 'all') :
            x=df.drop([yvar,subs], axis=1)
            print('selected all var for featres selection ')
        else : 
            x=df[xvar]
            print('not  all var for featres selection')


        y=df[yvar]
        y=np.ravel(y)
        if (selectVar=='RFE'): 
            if(modelRFE=='gbm'): model=GradientBoostingClassifier(n_estimators=10)
            if(modelRFE=='random'):model=RandomForestClassifier(n_estimators=10)

            if(modelRFE=='logreg'):model=BernoulliNB()
            if(modelRFE=='bays'):model=LogisticRegression()
            if(modelRFE=='DT'):model=tree.DecisionTreeClassifier()
            
            rfe=RFE(model)
            fit = rfe.fit(x,y)
            data={'Features':x.columns, 'Selected': fit.support_, 'Rank' : fit.ranking_}
            selectedTable=pd.DataFrame(data)
            booleandf = selectedTable.select_dtypes(include=[bool])
            booleanDictionary = {True: 'TRUE', False: 'FALSE'}
            for column in booleandf:
                selectedTable[column] = selectedTable[column].map(booleanDictionary)            

        else: 
            selectedTable=correlation(x,seuil)
            selectedTable['*'] =selectedTable.index 
            cols = selectedTable.columns.tolist()
            selectedTable = selectedTable[[cols[-1]] + cols[:-1]]         
        return selectedTable.to_dict('records')
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] >= threshold:
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset
                corrMatrix=dataset.corr()

    return (corrMatrix)

##############################################################################################################
############################ model report

@app.callback(Output('modelReport', 'rows'),
              [Input('table', 'rows'),
               Input('dropdownX', 'value'),
               Input('dropdownY', 'value'),
               Input('echSplit', 'value'),
               Input ('selectAll', 'value'),
               Input ('subs', 'value'),
               Input('model', 'value'),
               Input('report', 'n_clicks'),
               # MLP parameters
               Input('defaultMLP','value'),
               Input('neural','value'),
               Input('layers','value'),
               Input('stop','value'),
               # DT paramettres 
               Input('defaultDT','value'),
               Input('NEstimators','value'),
               Input('maxDepth','value'),
               Input('MinSampleSplit','value'),
               Input('MinSampleLeaf','value'),
               Input('maxFeature','value'),
               #selecte features
               Input('tableBestFeat', 'rows'),
               Input('selectVar', 'value'),
               Input('useSelectedFeature','value')
            ])

def modelReport (tablerows, xvar,yvar,ech,selectAll, subs, mod, button,
                 defaultMLP,neural,layers,stop,
                 defaultDT,NEstimators,maxDepth,MinSampleSplit,MinSampleLeaf,maxFeature,
                 tableBestFeat, selectVar, useSelectedFeature):
    
    if (button ==None):
        return [{}]
    else:
        moel=None 
        df =  pd.DataFrame(tablerows)
        y=df[yvar]
        y=np.ravel(y)
        if (useSelectedFeature== 'yes'):
            print('you choosed to work with selected features')    
            tableBestFeat=pd.DataFrame(tableBestFeat)
            if (selectVar=='RFE'):
                Features=tableBestFeat.loc[tableBestFeat['Selected'] == 'TRUE']['Features']
                x=df[Features]
                print('REF var loaded successfully')
            else:
                Features=tableBestFeat['*'].tolist()
                x=df[Features]
                print('corr deleted var loaded successfully')
        else: 
            print('you dicided to work with the inatial var')
            if (selectAll== 'all') :
                x=df.drop([yvar,subs], axis=1)
                print('selected all var loaded ')
            else : 
                x=df[xvar]
                print('not  all var  loaded')
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size =float(ech) ,  stratify=y)
        earlyStop=True
### log reg
        if (mod == 'logreg'):
            import time
            Title='Logistic regression'
            start_time = time.time() 
            logreg = LogisticRegression()
            model=logreg.fit(x_train,y_train)
            temps=time.time() - start_time
            # save for confusion matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMlogreg.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotLogreg.csv")

            print('logistic regression done')
### mlp            
        elif (mod =='mlp' ):
            import time
            start_time = time.time() 
            Title='Multi Layer Perceptron'
            if (defaultMLP=='T'):
                mlp=MLPClassifier()
            else:
                if(stop=='F'): earlStop=False
                if (float(layers)==1):
                    mlp=MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
                           beta_2=0.899, early_stopping=earlyStop , epsilon=1e-07,
                           hidden_layer_sizes=(int(neural)), learning_rate='invscaling',
                           learning_rate_init=0.1, max_iter=500, momentum=0.9,
                           nesterovs_momentum=True, power_t=0.5, random_state=None,
                           shuffle=True, solver='adam', tol=0.01, validation_fraction=0.1,
                           verbose=False, warm_start=False)
                elif (float(layers)==2):
                    mlp=MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
                           beta_2=0.899, early_stopping=earlyStop , epsilon=1e-07,
                           hidden_layer_sizes=(int(neural),int(neural)), learning_rate='invscaling',
                           learning_rate_init=0.1, max_iter=500, momentum=0.9,
                           nesterovs_momentum=True, power_t=0.5, random_state=None,
                           shuffle=True, solver='adam', tol=0.01, validation_fraction=0.1,
                           verbose=False, warm_start=False)                
                elif (float(layers)==3):
                    mlp=MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
                           beta_2=0.899, early_stopping=earlyStop , epsilon=1e-07,
                           hidden_layer_sizes=(int(neural),int(neural), int(neural)), learning_rate='invscaling',
                           learning_rate_init=0.1, max_iter=500, momentum=0.9,
                           nesterovs_momentum=True, power_t=0.5, random_state=None,
                           shuffle=True, solver='adam', tol=0.01, validation_fraction=0.1,
                           verbose=False, warm_start=False)
                

            model=mlp.fit(x_train, y_train)
            temps=time.time() - start_time
            # save for confusion matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMmlp.csv")            
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotMlp.csv")

            print('mlp done for model report')
### mlpACP            
        elif (mod=='mlpACP'):
            import time
            start_time = time.time()
            Title='MLP based on ACP'
            pca= PCA(n_components=37, whiten= True )
            pca.fit(x)
            X_pca = pca.transform(x)
            x_train, x_test, y_train, y_test = train_test_split(X_pca, y, test_size =0.3 )
            mlpACP = MLPClassifier(hidden_layer_sizes=(30),max_iter=500)
            model=mlpACP.fit(x_train,y_train)
            temps= time.time() - start_time
            ##save for confusion matrix 
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            df.to_csv(lien+"plotMlpACP.csv")

            confusion_matrix.to_csv(lien+"CMmlpACP.csv")
            print('mlpACP joblibd for model report')
### gbm            
        elif (mod =='gbm' ):
            Title='Gradient Boosting'
            import time
            start_time = time.time()
            if (defaultDT=='T'):
                gbm = GradientBoostingClassifier()
            else:
                if (maxFeature=='None'):
                    gbm = GradientBoostingClassifier(n_estimators=int(NEstimators), max_depth=int(maxDepth),
                                                     min_samples_split=int(MinSampleSplit),
                                                     min_samples_leaf=int(MinSampleLeaf))
                else:
                    gbm = GradientBoostingClassifier(n_estimators=int(NEstimators), max_depth=int(maxDepth),
                                                     min_samples_split=int(MinSampleSplit),
                                                     min_samples_leaf=int(MinSampleLeaf),
                                                     max_features=maxFeature)                
            model=gbm.fit(x_train, y_train)
            temps= time.time() - start_time
            # save for confusion matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMgbm.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            df.to_csv(lien+"plotGbm.csv")

            print('classification with GBM done successfully')
###random
        elif (mod =='random' ):
            Title= 'Rondom forest'
            import time
            start_time = time.time()
            if (defaultDT=='T'):
                random= RandomForestClassifier()
            else:
                if (maxFeature=='None'):
                    random= RandomForestClassifier(n_estimators=int(NEstimators), max_depth=int(maxDepth),
                                                     min_samples_split=int(MinSampleSplit),
                                                     min_samples_leaf=int(MinSampleLeaf))
                else:
                    random= RandomForestClassifier(n_estimators=int(NEstimators), max_depth=int(maxDepth),
                                                     min_samples_split=int(MinSampleSplit),
                                                     min_samples_leaf=int(MinSampleLeaf),
                                                     max_features=maxFeature)  
          
            model= random.fit(x_train, y_train)
            temps= time.time() - start_time
            # save for confusion matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMrandom.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotRandom.csv")

            print('classification with Random Forest done successfully')
### ababoost                
        elif(mod=='adaBoost'):
            Title='AdaBoost'
            import time
            start_time = time.time()
            if (defaultDT=='T'):
                ada = AdaBoostClassifier( )
            else:
                ada = AdaBoostClassifier(n_estimators=int(NEstimators) )
            model=ada.fit(x_train, y_train)
            temps= time.time() - start_time
            #save for  confusion matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMadaBoost.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotAdaBoost.csv")

            print('classification with adaBoost joblibd successfully')
### bagging
        elif(mod=='bagging'):
            Title='Bagging'
            import time
            start_time = time.time()
            if (defaultDT=='T'):
                ada = BaggingClassifier( )
            else:
                ada = BaggingClassifier(n_estimators=int(NEstimators) )
            temps= time.time() - start_time
            model=ada.fit(x_train, y_train)
            ## save  for confuision matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMbagging.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotBagging.csv")

            print('classification with Bagging joblibd successfully')
### DT ###            
        elif (mod=='DT'):
            import time
            start_time = time.time()
            Title='Decision Tree'
            if (defaultDT=='T'):
                DT=tree.DecisionTreeClassifier(min_samples_leaf=0.35,min_samples_split=0.01,max_depth=7)
            else:
                if (maxFeature=='None'):
                    DT=tree.DecisionTreeClassifier( max_depth=int(maxDepth),
                                                     min_samples_split=int(MinSampleSplit),
                                                     min_samples_leaf=int(MinSampleLeaf))
                else:
                    DT=tree.DecisionTreeClassifier( max_depth=int(maxDepth),
                                                     min_samples_split=int(MinSampleSplit),
                                                     min_samples_leaf=int(MinSampleLeaf),
                                                     max_features=maxFeature)
 
            model=DT.fit(x_train, y_train)
            temps= time.time() - start_time
            ## save for confusion Matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMDT.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotDT.csv")

            print('DT  done with specific parameter')
### bayes ####           
        elif(mod== 'bays'):
            import time
            start_time = time.time()
            Title='Naive Bays'
            bays = BernoulliNB()
            model= bays.fit(x_train, y_train)
            temps= time.time() - start_time
            print('Naive Byas  done')
            ## save for confusion matrix
            predict_y =   model.predict(x_test)
            confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
            confusion_matrix=pd.DataFrame(confusion_matrix)
            confusion_matrix.to_csv(lien+"CMbays.csv")
            ## save for plot
            fp, tp, threshold= metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])
            AUC= metrics.auc(fp, tp)
            data={ 'fp':  fp ,'tp': tp}
            df = pd.DataFrame(data)
            df.to_csv(lien+"plotBays.csv")

            print('classification with Niave Bays joblibd successfully')
### evaluation             
        AUC = metrics.auc(fp, tp)
        model_score =    model.score(x_test,y_test)

        #K-folds cross validaion
        cv_score = cross_validation.cross_val_score(  model, x_test,y_test, cv=3, scoring='roc_auc')   
        cv_mean=np.mean(cv_score)
        confusion_matrix = metrics.confusion_matrix(y_test, predict_y)
        TN= confusion_matrix[0, 0]                
        FP=confusion_matrix[0, 1]
        FN=confusion_matrix[1, 0]
        TP=confusion_matrix[1, 1] 
        #false negative rate error type I 
        fnr=FN/(FN+ TP)
        #False negaive rate error type II
        fpr=FP/(FP+TN)
        RSS = ((predict_y- y_test) ** 2).sum()
        k=x.shape[1]
        AIC= 2*k - 2*math.log(RSS )
        report=pd.DataFrame({'creteria':['Model','Time(seconds)','Accuracy','erreur I','erreurII' ,'AUC','CV ACU','AIC','som error']})

        report['Value']=[Title,temps,model_score,fnr,fpr,AUC, cv_mean,AIC,RSS]
        print('Report done successflly')
       
        ###
        return  report.to_dict('records')
    

#####################
@app.callback(Output('matrix', 'rows'),[Input('table', 'rows'),
               Input('dropdownX', 'value'),
               Input('dropdownY', 'value'),
               Input ('selectAll', 'value'),
               Input ('subs', 'value'),
               Input ('model', 'value'),
               Input ('echSplit', 'value'),
               Input ('ButtonMatrix', 'n_clicks'),
                #selecte features
               Input('tableBestFeat', 'rows'),
               Input('selectVar', 'value'),
               Input('useSelectedFeature','value')
            ])
def matrixDef (tablerows, xvar,yvar, selectAll, subs, mod,ech, button,
               tableBestFeat, selectVar, useSelectedFeature):
    if (button ==None):
        return [{}]
    else:
        model=None
        df =  pd.DataFrame(tablerows)
        y=df[yvar]
        y=np.ravel(y)
        if (useSelectedFeature== 'yes'):
            print('you choosed to work with selected features')    
            tableBestFeat=pd.DataFrame(tableBestFeat)
            if (selectVar=='RFE'):
                Features=tableBestFeat.loc[tableBestFeat['Selected'] == 'TRUE']['Features']
                x=df[Features]
                print('REF var loaded successfully')
            else:
                Features=tableBestFeat['*'].tolist()
                x=df[Features]
                print('corr deleted var loaded successfully')
        else: 
            print('you dicided to work with the inatial var')
            if (selectAll== 'all') :
                x=df.drop([yvar,subs], axis=1)
                print('selected all var loaded  to compute Confucion Matirx')
            else : 
                x=df[xvar]
                print('not all var loaded  to compute Confucion Matirx')

        if(mod == 'logreg'):
            titre=' logistic Regression'
            m= pd.read_csv(lien+"CMlogreg.csv",sep=',')
            print('confusion martix logreg done')
            
        elif (mod=='mlp' ):
            titre='MLPerceptron'
            m= pd.read_csv(lien+"CMmlp.csv",sep=',')
            print('confusion martix mlp done')
            
        elif (mod=='gbm'):
            titre='Gradient Boosting'
            m= pd.read_csv(lien+"CMgbm.csv",sep=',')
            print('confusion martix gbm done')
        elif (mod=='random'):   
            titre='Random Forest'
            m= pd.read_csv(lien+"CMrandom.csv",sep=',')
            print('confusion martix random forest done')
            
        elif (mod=='adaBoost'):    
            titre='AdaBoost Tree'
            m= pd.read_csv(lien+"CMada.csv",sep=',')

            print('confusion martix adaBoost  done')        
        elif (mod=='bagging'):    
            titre='bagging'
            m= pd.read_csv(lien+"CMbagging.csv",sep=',')
            print('confusion martix baggib Tree done')
        elif (mod=='DT'):    
            titre= 'Decision Tree'
            m= pd.read_csv(lien+"CMDT.csv",sep=',')
            print('confusion martix DT done')
        elif('mlpACP'== mod):
            titre='MLPerceptron ACP'
            m= pd.read_csv(lien+"CMmlpACP.csv",sep=',')
            print('confucion matrix mlpACP done')
            
        elif('bays'== mod):
            titre='Bayes'
            m= pd.read_csv(lien+"CMbays.csv",sep=',')
            print('confusion martix bays done')
        data={titre:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
        data=pd.DataFrame(data)
        print(data)
        return data.to_dict('records')
    
#########################################################################################################
#####################  roc plot



#####################
@app.callback(Output('my-graph', 'figure'),
              [Input('table', 'rows'),
               Input('dropdownX', 'value'),
               Input('dropdownY', 'value'),
               Input ('selectAll', 'value'),
               Input ('subs', 'value'),
               Input ('model', 'value'),
               Input ('echSplit', 'value'),
               Input ('roc','n_clicks'),
                #selecte features
               Input('tableBestFeat', 'rows'),
               Input('selectVar', 'value'),
               Input('useSelectedFeature','value')
              ])

def RocPlot (tablerows, xvar,yvar, selectAll, subs, mod , ech, button,
               tableBestFeat, selectVar, useSelectedFeature):
    if (button ==None):
        return [{}]
    else:
        model=None
        df =  pd.DataFrame(tablerows)
        y=df[yvar]
        y=np.ravel(y)
        if (useSelectedFeature== 'yes'):
            print('you choosed to work with selected features')    
            tableBestFeat=pd.DataFrame(tableBestFeat)
            if (selectVar=='RFE'):
                Features=tableBestFeat.loc[tableBestFeat['Selected'] == 'TRUE']['Features']
                x=df[Features]
                print('REF var loaded successfully')
            else:
                Features=tableBestFeat['*'].tolist()
                x=df[Features]
                print('corr deleted var loaded successfully')
        else: 
            print('you dicided to work with the inatial var')
            if (selectAll== 'all') :
                x=df.drop([yvar,subs], axis=1)
                print('selected all var loaded  to compute Confucion Matirx')
            else : 
                x=df[xvar]
                print('not all var loaded  to compute Confucion Matirx')
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size =float(ech) ,  stratify=y)

        if('logreg'== mod):
            Title='Logistic Regression'
            m= pd.read_csv(lien+"plotLogreg.csv",sep=',')
            print('Logistic reg model loaded for plot')

        elif('mlp'== mod):
            Title='Multi Layer perceptron'
            m= pd.read_csv(lien+"plotMlp.csv",sep=',')
            print('mlp model loaded for plot')

        elif('gbm'== mod):
            Title='Gradient Boosting'
            m= pd.read_csv(lien+"plotGbm.csv",sep=',')
            print('gbm model loaded for plot')

        elif('random'== mod):
            Title='Random Forest'
            m= pd.read_csv(lien+"plotRandom.csv",sep=',')
            print('random model loaded for plot')

        elif('adaBoost'== mod):
            Title='AdaBoost'
            m= pd.read_csv(lien+"plotAdaBoost.csv",sep=',')
            print('AdaBoost loaded for plot')

        elif('bagging'== mod):
            Title='Bagging'
            m= pd.read_csv(lien+"plotBagging.csv",sep=',')
            print('bagging loaded for plot')
            
        elif('DT'== mod):
            Title='Decision Tree'
            m= pd.read_csv(lien+"plotDT.csv",sep=',')
            print('DT loaded for plot')

        elif('mlpACP'== mod):
            Title='mlp based on ACP'
            m= pd.read_csv(lien+"plotMlpACP.csv",sep=',')
            print('mlpACP loaded for plot')            
            
        else: 
            Title='Naive Bays'
            m= pd.read_csv(lien+"plotBays.csv",sep=',')
            print(' Bays loaded model  for plot')

        fp=m.fp
        tp=m.tp
        AUC= metrics.auc(fp, tp)
        lw = 2
        trace1 = go.Scatter(x=fp, y=tp, 
                            mode='lines', 
                            line=dict(color='darkorange', width=lw),
                            name='ROC curve (area = %0.2f)' % AUC )
        trace2 = go.Scatter(x=[0, 1], y=[0, 1], 
                            mode='lines', 
                            line=dict(color='navy', width=lw, dash='dash'),
                            showlegend=False)
        layout = go.Layout(title=Title,
                            xaxis=dict(title='False Positive Rate'),
                            yaxis=dict(title='True Positive Rate'))
        print('plot done')
        return{ 'data': [trace1, trace2], 'layout': layout }
        dcc.Graph(id='multiROC'),
#############################################################################################################
###### multi roc

##########
@app.callback(Output('multiROC', 'figure'),
              [Input('table', 'rows'),
               Input('dropdownX', 'value'),
               Input('dropdownY', 'value'),
               Input ('selectAll', 'value'),
               Input ('subs', 'value'),
               Input ('echSplit', 'value'),
               Input ('Mroc','n_clicks'),
               Input ('Multimodel', 'values'),
                #selecte features
               Input('tableBestFeat', 'rows'),
               Input('selectVar', 'value'),
               Input('useSelectedFeature','value')
               
              ])

def MultiROCPlot (tablerows, xvar,yvar, selectAll, subs, ech, button, mod,
               tableBestFeat, selectVar, useSelectedFeature):
    lw=2
    trace = go.Scatter(x=[0, 1], y=[0, 1], 
                        mode='lines', 
                        line=dict(color='navy', width=lw, dash='dash'),
                        showlegend=False)
    layout = go.Layout(title='Compare last generated models',
                        xaxis=dict(title='False Positive Rate'),
                        yaxis=dict(title='True Positive Rate'))
    if (button ==None):
        return { 'data': [trace], 'layout': layout }
    else:
        if ('logreg' in mod): 
            m= pd.read_csv(lien+"plotLogreg.csv",sep=',')
            print(' logreg loaded model  for multi  plot')
            fp_logreg=m.fp
            tp_logreg=m.tp
            AUC_logreg= metrics.auc(fp_logreg, tp_logreg)            
            trace1 = go.Scatter(x=fp_logreg, y=tp_logreg, 
                                mode='lines', 
                                line=dict(color='darkorange', width=lw),
                                name='Logreg ROC  curve (area = %0.2f)' % AUC_logreg )
        else :  trace1=trace
        if ('mlp' in mod): 
            m= pd.read_csv(lien+"plotMlp.csv",sep=',')
            fp_mlp=m.fp
            tp_mlp=m.tp
            AUC_mlp= metrics.auc(fp_mlp, tp_mlp)            
            trace2 = go.Scatter(x=fp_mlp, y=tp_mlp, 
                                mode='lines', 
                                line=dict(color='orange', width=lw),
                                name='MLP ROC curve (area = %0.2f)' % AUC_mlp )
        else: trace2=trace
        if ('gbm'in mod):
            print(' gbm loaded model  for multi  plot')
            m= pd.read_csv(lien+"plotGbm.csv",sep=',')
            fp_gbm=m.fp
            tp_gbm=m.tp
            AUC_gbm= metrics.auc(fp_gbm, tp_gbm)                        
            trace3 = go.Scatter(x=fp_gbm, y=tp_gbm, 
                                mode='lines', 
                                line=dict(color='pink', width=lw),
                                name=' GBM ROC curve (area = %0.2f)' % AUC_gbm )
        else: trace3=trace
        if('random' in mod):
            m= pd.read_csv(lien+"plotRandom.csv",sep=',')
            print(' random loaded model  for multi  plot')
            fp_random=m.fp
            tp_random=m.tp
            AUC_random= metrics.auc(fp_random, tp_random)            

            trace4 = go.Scatter(x=fp_random, y=tp_random, 
                                mode='lines', 
                                line=dict(color='blue', width=lw),
                                name='Random forest ROC curve (area = %0.2f)' % AUC_random )
        else: trace4=trace
        if('adaBoost' in mod):
            m= pd.read_csv(lien+"plotAdaBoost.csv",sep=',')
            print(' adaBoost loaded model  for multi  plot')
            fp_adaBoost=m.fp
            tp_adaBoost=m.tp
            AUC_adaBoost= metrics.auc(fp_adaBoost, tp_adaBoost)            

            trace5 = go.Scatter(x=fp_adaBoost, y=tp_adaBoost, 
                                mode='lines', 
                                line=dict(color='#ff66d9', width=lw),
                                name='AdaBoost forest ROC curve (area = %0.2f)' % AUC_adaBoost )
        else: trace5=trace
        if('bagging' in mod):
            m= pd.read_csv(lien+"plotBagging.csv",sep=',')
            print(' bagging loaded model  for multi  plot')
            fp_bagging=m.fp
            tp_bagging=m.tp
            AUC_bagging= metrics.auc(fp_bagging, tp_bagging)            

            trace6 = go.Scatter(x=fp_bagging, y=tp_bagging, 
                                mode='lines', 
                                line=dict(color='#00e600', width=lw),
                                name='Bagging forest ROC curve (area = %0.2f)' % AUC_bagging )
        else: trace6=trace
        if('DT' in mod):
            m= pd.read_csv(lien+"plotDT.csv",sep=',')
            print(' DT loaded model  for multi  plot')
            fp_DT=m.fp
            tp_DT=m.tp
            AUC_DT= metrics.auc(fp_DT, tp_DT)            

            trace7 = go.Scatter(x=fp_DT, y=tp_DT, 
                                mode='lines', 
                                line=dict(color='#ff3300', width=lw),
                                name='DT forest ROC curve (area = %0.2f)' % AUC_DT )
        else: trace7=trace
        if('mlpACP' in mod):
            m= pd.read_csv(lien+"plotMlpACP.csv",sep=',')
            print(' mlpACP loaded model  for multi  plot')
            fp_mlpACP=m.fp
            tp_mlpACP=m.tp
            AUC_mlpACP= metrics.auc(fp_mlpACP, tp_mlpACP)            

            trace8 = go.Scatter(x=fp_mlpACP, y=tp_mlpACP, 
                                mode='lines', 
                                line=dict(color='#00cccc', width=lw),
                                name='MlpACP forest ROC curve (area = %0.2f)' % AUC_mlpACP )
        else: trace8=trace

        if ('bays' in mod):
            m= pd.read_csv(lien+"plotBays.csv",sep=',')
            print(' bays loaded model  for multi  plot')
            fp_bays=m.fp
            tp_bays=m.tp
            AUC_bays= metrics.auc(fp_bays, tp_bays)            
            trace9 = go.Scatter(x=fp_bays, y=tp_bays, 
                                mode='lines', 
                                line=dict(color='deeppink', width=lw),
                                name=' Bays ROC curve (area = %0.2f)' % AUC_bays )
        else : trace9= trace
        
        layout = go.Layout(title='Receiver operating characteristic example',
                            xaxis=dict(title='False Positive Rate'),
                            yaxis=dict(title='True Positive Rate'))
        print(' multi plot done')
        return{ 'data': [trace1, trace2, trace3, trace4, trace5,trace6,trace7,trace8,trace9, trace], 'layout': layout }
    
###########################################################################################################
### compare table     

@app.callback(Output('compareTable', 'rows'),
              [
               Input ('Mroc','n_clicks'),   
               Input ('Multimodel', 'values'),
              ])

def compareTable (button,mod):

    report=pd.DataFrame({'Evaluation metric':['Model','AUC','Accuracy','erreur I','erreurII' ]})
    if (button ==None):
        return [{}]
    else:
### log reg        
        if ('logreg' in mod): 
            titre_logreg='Logistic Regression'
            m= pd.read_csv(lien+"plotLogreg.csv",sep=',')
            fp_logreg=m.fp
            tp_logreg=m.tp
            AUC_logreg= metrics.auc(fp_logreg, tp_logreg)
            m= pd.read_csv(lien+"CMlogreg.csv",sep=',')
            data={titre_logreg:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_logreg =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_logreg=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_logreg=FP/(FP+TN)
            report[titre_logreg]=[titre_logreg,AUC_logreg,model_score_logreg,fnr_logreg,fpr_logreg]
            print('compare table logreg done ')
        if ('gbm' in mod): 
            titre_gbm='Gradient Boosting'
            m= pd.read_csv(lien+"plotGbm.csv",sep=',')
            fp_gbm=m.fp
            tp_gbm=m.tp
            AUC_gbm= metrics.auc(fp_gbm, tp_gbm)
            m= pd.read_csv(lien+"CMgbm.csv",sep=',')
            data={titre_gbm:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_gbm =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_gbm=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_gbm=FP/(FP+TN)
            report[titre_gbm]=[titre_gbm,AUC_gbm,model_score_gbm,fnr_gbm,fpr_gbm]
            print('compare table gbm done ')

        if ('DT' in mod): 
            titre_DT='Decision Tree'
            m= pd.read_csv(lien+"plotDT.csv",sep=',')
            fp_DT=m.fp
            tp_DT=m.tp
            AUC_DT= metrics.auc(fp_DT, tp_DT)

            m= pd.read_csv(lien+"CMDT.csv",sep=',')
            data={titre_DT:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_DT =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_DT=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_DT=FP/(FP+TN)
            report[titre_DT]=[titre_DT,AUC_DT,model_score_DT,fnr_DT,fpr_DT]
        if ('adaBoost' in mod): 
            titre_adaBoost='AdaBoost Tree'
            m= pd.read_csv(lien+"plotAdaBoost.csv",sep=',')
            fp_adaBoost=m.fp
            tp_adaBoost=m.tp
            AUC_adaBoost= metrics.auc(fp_adaBoost, tp_adaBoost)
            m= pd.read_csv(lien+"CMadaBoost.csv",sep=',')
            data={titre_adaBoost:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_adaBoost =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_adaBoost=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_adaBoost=FP/(FP+TN)
            report[titre_adaBoost]=[titre_adaBoost,AUC_adaBoost,model_score_adaBoost,fnr_adaBoost,fpr_adaBoost]
        if ('random' in mod): 
            titre_random='Random Forest'
            m= pd.read_csv(lien+"plotRandom.csv",sep=',')
            fp_random=m.fp
            tp_random=m.tp
            AUC_random= metrics.auc(fp_random, tp_random)
            m= pd.read_csv(lien+"CMrandom.csv",sep=',')
            data={titre_random:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_random =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_random=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_random=FP/(FP+TN)
            report[titre_random]=[titre_random,AUC_random,model_score_random,fnr_random,fpr_random]
        if ('bagging' in mod): 
            titre_bagging='Bagging Tree'
            m= pd.read_csv(lien+"plotBagging.csv",sep=',')
            fp_bagging=m.fp
            tp_bagging=m.tp
            AUC_bagging= metrics.auc(fp_bagging, tp_bagging)
            m= pd.read_csv(lien+"CMbagging.csv",sep=',')
            data={titre_bagging:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_bagging =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_bagging=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_bagging=FP/(FP+TN)
            report[titre_bagging]=[titre_bagging,AUC_bagging,model_score_bagging,fnr_bagging,fpr_bagging]
        if ('mlp' in mod): 
            titre_mlp='ML Perceptrons'
            m= pd.read_csv(lien+"plotMlp.csv",sep=',')
            fp_mlp=m.fp
            tp_mlp=m.tp
            AUC_mlp= metrics.auc(fp_mlp, tp_mlp)
            m= pd.read_csv(lien+"CMmlp.csv",sep=',')
            data={titre_mlp:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_mlp =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_mlp=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_mlp=FP/(FP+TN)
            report[titre_mlp]=[titre_mlp,AUC_mlp,model_score_mlp,fnr_mlp,fpr_mlp]
        if ('mlpACP' in mod): 
            titre_mlpACP='ML Perceptrons ACP'
            m= pd.read_csv(lien+"plotMlpACP.csv",sep=',')
            fp_mlpACP=m.fp
            tp_mlpACP=m.tp
            AUC_mlpACP= metrics.auc(fp_mlpACP, tp_mlpACP)
            m= pd.read_csv(lien+"CMmlpACP.csv",sep=',')
            data={titre_mlpACP:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_mlpACP =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_mlpACP=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_mlpACP=FP/(FP+TN)
            report[titre_mlpACP]=[titre_mlpACP,AUC_mlpACP,model_score_mlpACP,fnr_mlpACP,fpr_mlpACP]
        if ('bays' in mod): 
            titre_bays='bays'
            m= pd.read_csv(lien+"plotBays.csv",sep=',')
            fp_bays=m.fp
            tp_bays=m.tp
            AUC_bays= metrics.auc(fp_bays, tp_bays)
            m= pd.read_csv(lien+"CMbays.csv",sep=',')
            data={titre_bays:['actuel 0','actuel 1'],'predicted 0':[m.iloc[0]['0'],m.iloc[1]['0'] ], 'predicted 1':[m.iloc[0]['1'],m.iloc[1]['1'] ] }
            TN= m.iloc[0]['0']   
            FP=m.iloc[0]['1']
            FN=m.iloc[1]['0']
            TP=m.iloc[1]['1']
            ## accuracy
            model_score_bays =(TP+TN)/(TP+TN+FP+FN)
            #false negative rate error type I 
            fnr_bays=FN/(FN+ TP)
            #False negaive rate error type II
            fpr_bays=FP/(FP+TN)
            report[titre_bays]=[titre_bays,AUC_bays,model_score_bays,fnr_bays,fpr_bays]
            print('compare table  bays added')
            


       
        ###
        return  report.to_dict('records')    

###########################################################################################################
########### make a prediction 
@app.callback(Output('predict', 'rows'),
              [Input('upload_pred', 'contents'),
               Input('upload_pred', 'filename'),
               Input('dropdownY', 'value'),
               Input('dropdownX','value'),
               Input ('subs', 'value'),
               Input ('model', 'value'),
               Input ('selectAll', 'value'),
                #selecte features
               Input('tableBestFeat', 'rows'),
               Input('selectVar', 'value'),
               Input('useSelectedFeature','value')
               
              ])
def predict_output(contents, filename,yvar,xvar, subs,mod, selectAll,
               tableBestFeat, selectVar, useSelectedFeature):
    if contents is not None:
        df = parse_contents(contents, filename)
        if df is not None:
            ####
            if (useSelectedFeature== 'yes'):
                print('you choosed to work with selected features')    
                tableBestFeat=pd.DataFrame(tableBestFeat)
                if (selectVar=='RFE'):
                    Features=tableBestFeat.loc[tableBestFeat['Selected'] == 'TRUE']['Features']
                    x=df[Features]
                    print('REF var loaded successfully')
                else:
                    Features=tableBestFeat['*'].tolist()
                    x=df[Features]
                    print('corr deleted var loaded successfully')
            else: 
                print('you dicided to work with the inatial var')
                if (selectAll== 'all') :
                    x=df.drop([subs], axis=1)
                    print(' all var loaded  to compute prediction')
                else : 
                    x=df[xvar]
                    print('not all var loaded  to compute prediction')

            if('logreg'== mod):
                with open("python_logreg_model.pkl", "rb") as file_handler:
                    model = joblib.load(file_handler)
                print('Logistic reg model loaded for prediction')

            elif('mlp'== mod):
                with open("python_mlp_model.pkl", "rb") as file_handler:
                    model = joblib.load(file_handler)
                print('mlp model loaded for prediction')

            elif('gbm'== mod):
                with open("python_gbm_model.pkl", "rb") as file_handler:
                    model = joblib.load(file_handler)
                print('gbm model loaded for prediction')

            elif('random'== mod):
                with open("python_random_model.pkl", "rb") as file_handler:
                    model = joblib.load(file_handler)
                print('random model loaded for prediction')


            else: 
                with open("python_bays_model.pkl", "rb") as file_handler:
                    model = joblib.load(file_handler)
                print(' Bays loaded model  for prediction')
            
            ####

            pre=pd.DataFrame(model.predict(x))
            p=pd.concat([pre,df[subs]],axis=1 )
            print('prediction done')
            return p.to_dict('records')            
        else:
            return [{}]
    else:
        return [{}]
#######
@app.callback( Output('downloadResult', 'children'),
              [ Input('predict','rows'),
                Input('download', 'n_clicks')],
                [dash.dependencies.State('url','value')]
               
              )
def downloadTbale(table, button, url):
    if (button ==0) : 
        return [{}]
    else:
        fileName=str(url)+'/predict.csv'
        table=pd.DataFrame(table)
        table.to_csv(fileName, sep=';', encoding='utf-8')
        return 'Prediction table has been dowloadded successfully in : "{}"' .format(fileName)


    
app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})

if __name__ == '__main__':
    app.run_server(debug=False)


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Sep/2018 20:11:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:22] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:22] "GET /_dash-component-suites/dash_renderer/react-dom@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:22] "GET /_dash-component-suites/dash_html_components/bundle.js?v=0.10.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:23] "GET /_dash-component-suites/dash_core_components/plotly-1.31.0.min.js?v=0.15.0rc1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:23] "GET /_dash-component-suites/dash_core_components/bundle.js?v=0.15.0rc1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:23] "GET /_dash-component-suites/dash_table_experiments/bundle.js?v=0.6.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:23] "GET /_dash-component-suites/dash_renderer/bundle.js?v=0.12.1 HTTP/1.1" 200 -
127.0

contenant is none !


127.0.0.1 - - [04/Sep/2018 20:11:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:26] "POST /_dash-update-component HTTP/1.1" 200 -


updating... dff empty?: True
updating... dff empty?: True
updating... dff empty?: True


127.0.0.1 - - [04/Sep/2018 20:11:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2018 20:11:27] "POST /_dash-update-component HTTP/1.1" 200 -
[2018-09-04 20:11:27,602] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\DELL\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\DELL\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\DELL\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\DELL\Anaconda3\lib\site-packages\flask\

...

Report done successflly


127.0.0.1 - - [04/Sep/2018 20:13:50] "POST /_dash-update-component HTTP/1.1" 200 -


compare table gbm done 


127.0.0.1 - - [04/Sep/2018 20:13:51] "POST /_dash-update-component HTTP/1.1" 200 -


 gbm loaded model  for multi  plot
 multi plot done
